In [2]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

# Visualization
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa.display
import seaborn as sns

#Neuronal networks
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.models import load_model
import keras
import utils

Using TensorFlow backend.


### Changing MP3 to WAV

All tracks should be change to wav format in order to work easily whit python

In [5]:

path = '.\\data\\fma_small'
utils.all_mp3_to_wav(path)

.\data\fma_small
.\data\fma_small\000
.\data\fma_small\001


KeyboardInterrupt: 

### Reading metadata

In [132]:
AUDIO_DIR='.\\data\\fma_small'
META_DIR='.\\data\\fma_metadata'

features = utils.load(META_DIR+'\\features.csv')
echonest = utils.load(META_DIR+'\\echonest.csv')
genres = utils.load(META_DIR+'\\genres.csv')
tracks = utils.load(META_DIR+'\\tracks.csv')
#raw_tracks = utils.load(META_DIR+'\\raw_tracks.csv')
artists = utils.load(META_DIR+'\\raw_artists.csv')



C:\Users\User_01\Documents\Master\DeepLearning\Proyecto\fma\utils.py:225: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


### Get small subset of data

In [195]:
# Creating the subset 
small = tracks[tracks['set', 'subset'] <= 'small']

In [197]:

features



feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   
26           -0.699535 -0.684158  0.048825  0.042659 -0.818967 -0.917123   
30           -0.721487 -0.848560  0.890904  0.088620 -0.445513 -1.271170   
46           -0.119708 -0.858814  2.362546  0.106584 -1.315912 -1.220354   
48           -1.054053  0.932339  0.528064 -1.035338 -1.000682 -1.119304   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
135           0.978402  0.623572  1.129261 -1.043970 -1.127288 -0.032642   
136           0.915001 -0.643476 -0.460507 -0.530701 -0.364460 -0.226860   
137           1.607191  1.199649  0.745765  1.430835 -0.155455  0.768487   
138           1.501403  0.957514  1.276685  0.640389  0.442936  0.845533   
139          -0.020869  0.432330  0.331278  0.829845  2.625593  2.005660   
140           0.533579 -0.623885 -1.086205 -1.081079 -0.765151 -0.072282   
141           0.172898 -0.284804 -1.169662 -1.062855 -0.706868 -0.708281   
142          -0.581270  3.199484  1.298346 -0.681253 -0.935093 -0.960304   
144           0.121207 -0.025308  0.541327  0.967687  1.082456  1.675841   
145           0.954054  0.315324  0.015383 -0.023933  0.086671  1.279086   
146          -0.763713 -0.342105 -0.273965  0.729047 -0.649717  0.109572   
147           0.011158  0.069270  1.552282 -0.251362  0.158631  0.071886   
148          -0.453950 -0.922267 -0.845767 -1.059609 -0.164252 -0.428238   
149          -0.386733 -0.545598 -0.638649 -0.669786 -0.522530 -0.809405   
150          -0.083043  0.142131 -0.466703  1.447829 -0.324532  0.098253   
151           2.679700  1.564581  1.628766  1.574172  2.461945  4.033635   
152          -0.510457  1.649074  0.340847  0.389235 -0.371745  1.578730   
153           2.437779  0.886211  2.899974  1.726185  1.248919  0.676350   
154          40.440720 -0.251259  0.685472 -0.670811 -1.223815  1.275721   
155           0.161195  1.580878 -0.200231  0.216993  0.196968 -0.442953   
...                ...       ...       ...       ...       ...       ...   
155290       -1.256915 -1.194825 -1.353125 -0.532530 -1.404040 -0.995579   
155291       -1.543836 -1.320838 -1.199663 -0.006909 -1.160004 -0.326463   
155292       -0.955190 -0.879837  0.480942  2.266221 -0.899861 -0.904715   
155293        0.116289 -0.728864 -0.404994  0.000208 -0.377840 -0.198419   
155294       -1.052930 -0.875463  0.108331 -1.031980 -1.257848 -1.403183   
155295       -1.855166 -1.843082  6.121086  4.632593  5.402946  8.081429   
155296       -1.181767 -0.953902 -0.767223 -0.960635 -1.507767 -0.655886   
155297       -0.007668  0.104604 -0.511487  0.857308  0.266202 -0.355186   
155298       -0.956019 -0.961210 -0.447053 -0.689531 -0.633071 -0.430116   
155299       -0.378812 -0.692087 -0.869693 -0.862368  0.000357  0.103398   
155300       -0.590855 -0.228609 -0.592955 -0.585120 -0.613510  1.489933   
155301       -0.749575 -0.742979 -0.218289  0.134936  0.882519  0.471344   
155302       -0.732580 -0.203825 -0.883129 -0.393307 -0.742957 -0.767015   
155303       -0.846720 -1.161092 -0.724193 -0.604951 -1.066579 -1.133118   
155304       -0.420769 -0.099274 -0.251443  0.932526  0.313478  0.347822   
155305       -0.832872 -0.809870 -0.601314 -0.343021 -0.922507 -0.550964   
155306        0.644815 -0.879404 -1.149232 -0.655986 -0.848881 -0.672240   
155307       -0.446937  1.235004 -0.250855 -0.61